In [5]:
import pandas as pd
import numpy as np
import fastf1
import datetime
fastf1.Cache.enable_cache('cache') 

In [6]:
def find_fastest_laps(laps, cols, index_cols, lap_num):
    
    laps = laps.sort_values('LapTime')
    fastest = laps.groupby('DriverNumber').nth(0).reset_index()

    fastest_time = {}
    added_cols = []
    
    
    for col in cols:
        fastest_time[col] = laps[col].min()
        
        added_cols.append(col + 'Percent')
       # added_cols.append(col[:-4] + 'Position')
        # fastest[col[:-4] + 'Percent'] = (fastest[col] - fastest_time) / fastest_time
        # total_cols.append(col[:-4] + 'Percent')
    
    # dummies = pd.get_dummies(fastest['Compound'])
    # #dummies.columns = dummies.columns + '_1'
    # fastest = pd.concat([fastest, dummies], axis = 1)
    
    fastest = fastest[index_cols]
    

    # top_cols = ['Sector1Time', 'Sector2Time', 'Sector3Time']
    # fastest = fastest.merge(laps.groupby('DriverNumber').min()[top_cols].reset_index(), on=['DriverNumber'], how = 'outer')
    for i in range(0,lap_num):
        laps_i = laps.groupby('DriverNumber').nth(i).reset_index()

        for col in cols:
            laps_i[col + 'Percent'] = (laps_i[col] - fastest_time[col]) / fastest_time[col]
            #laps_i[col[:-4] + 'Position'] = laps_i.sort_values(col).index
        # laps_i['LapPercent'] = (laps_i['LapTime'] - fastest_lap) / fastest_lap
        fastest = fastest.merge(laps_i[index_cols + cols + added_cols], on=index_cols, how = 'outer', suffixes=('', '_' + str(i)))
    return fastest

In [7]:
def load_practice(event, type, year):
    
    practice1 = fastf1.get_session(year, event, 'Practice 1')
    practice1.load(laps=True,messages=True)
    try:
        laps1 = practice1.laps
        cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time','TyreLife','SpeedI1','SpeedI2','SpeedFL']
        index_cols = ['DriverNumber', 'Team', 'Driver']
        practice1 = find_fastest_laps(laps1, cols, index_cols, 2)
        practice2 = practice1
        if type == 'conventional':
            practice2 = fastf1.get_session(year, event, 'Practice 2')
            practice2.load(laps=True,messages=True)
            laps2 = practice2.laps
            practice2 = find_fastest_laps(laps2, cols, index_cols, 2)

        practice_data = practice1.merge(practice2, on= index_cols, how = 'outer', suffixes=('_1', '_2'))
        return practice_data
    except:
         return pd.DataFrame()
   
def load_qualifying(event, year):
        qualifying = fastf1.get_session(year, event, 'Q')
        qualifying.load()
        try:
            results = qualifying.results

            results['FastestLap'] = results[['Q1', 'Q2', 'Q3']].apply(lambda x: min(x), axis = 1)

            fastest_lap_q = results['FastestLap'].min()
            results['LapTimePercent'] = (results['FastestLap'] - fastest_lap_q)/ fastest_lap_q
            results['Team'] = results['TeamName']
            return results[['DriverNumber', 'Position', 'LapTimePercent', 'Team']]
        except:
             return pd.DataFrame()


In [8]:
years = [2017, 2018, 2019,2020, 2021, 2022, 2023]
years = [2023]

intitial_data = pd.DataFrame()
#prev_qualifying_data = pd.DataFrame()
for year in years:
    schedule = fastf1.get_event_schedule(year)
    for index, row in schedule.iterrows():
        if row['EventFormat'] in ['conventional']:
            practice_data = load_practice(row['EventName'], row['EventFormat'], year)
            qualifying_data = load_qualifying(row['EventName'], year)
            if 'DriverNumber' in practice_data.columns and 'DriverNumber' in qualifying_data.columns:
                #if 'DriverNumber' in prev_qualifying_data.columns:
                    # qualifying_data_full = qualifying_data.merge(prev_qualifying_data, on = 'DriverNumber', suffixes=('_curr', '_prev'))
                full_data = practice_data.merge(qualifying_data, on=['DriverNumber', 'Team'])
                for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
                    full_data[col] = row[col]
                intitial_data = pd.concat([intitial_data, full_data], axis = 0, ignore_index=True)
                # prev_qualifying_data = qualifying_data

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, re

In [9]:
data = pd.DataFrame(intitial_data)


In [10]:
prev_range = 5

In [11]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'TyreLife_1', 'SpeedI1_1',
       'SpeedI2_1', 'SpeedFL_1', 'LapTimePercent_1', 'Sector1TimePercent_1',
       'Sector2TimePercent_1', 'Sector3TimePercent_1', 'TyreLifePercent_1',
       'SpeedI1Percent_1', 'SpeedI2Percent_1', 'SpeedFLPercent_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'TyreLife_1_1', 'SpeedI1_1_1', 'SpeedI2_1_1', 'SpeedFL_1_1',
       'LapTimePercent_1_1', 'Sector1TimePercent_1_1',
       'Sector2TimePercent_1_1', 'Sector3TimePercent_1_1',
       'TyreLifePercent_1_1', 'SpeedI1Percent_1_1', 'SpeedI2Percent_1_1',
       'SpeedFLPercent_1_1', 'LapTime_2', 'Sector1Time_2', 'Sector2Time_2',
       'Sector3Time_2', 'TyreLife_2', 'SpeedI1_2', 'SpeedI2_2', 'SpeedFL_2',
       'LapTimePercent_2', 'Sector1TimePercent_2', 'Sector2TimePercent_2',
       'Sector3TimePercent_2', 'TyreLifePercent_2', 'SpeedI1Percent_2',
      

In [12]:
team_cols = []
individual_cols = []


for i in range(prev_range):
    data['PrevLapTimePercent' + str(i+1)] = data.groupby('DriverNumber')['LapTimePercent'].shift(i+1)
    individual_cols.append('PrevLapTimePercent' + str(i+1))
    # data['PrevPosition' + str(i+1)] = data.groupby('DriverNumber')['Position'].shift(i+1)
    prev = data.groupby(['Team', 'OfficialEventName', 'EventDate'])[['Position', 'LapTimePercent']].mean().reset_index().sort_values('EventDate').groupby('Team').shift(i+1)
    prev.index = data.groupby(['Team', 'OfficialEventName', 'EventDate'])[['Position', 'LapTimePercent']].mean().reset_index(2).sort_values('EventDate').index
    prev = prev[['Position', 'LapTimePercent']].reset_index()
    data = data.merge(prev, on = ['OfficialEventName', 'Team'], suffixes=('', 'PrevTeam' + str(i+1)))
    team_cols.append('LapTimePercentPrevTeam' + str(i+1))
    #team_cols.append('PositionPrevTeam' + str(i+1))

In [13]:
def cap_outliers(row):
    median = np.median(row)
    return np.clip(row, median-0.02, median+0.02)

In [14]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'TyreLife_1', 'SpeedI1_1',
       'SpeedI2_1', 'SpeedFL_1', 'LapTimePercent_1', 'Sector1TimePercent_1',
       'Sector2TimePercent_1', 'Sector3TimePercent_1', 'TyreLifePercent_1',
       'SpeedI1Percent_1', 'SpeedI2Percent_1', 'SpeedFLPercent_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'TyreLife_1_1', 'SpeedI1_1_1', 'SpeedI2_1_1', 'SpeedFL_1_1',
       'LapTimePercent_1_1', 'Sector1TimePercent_1_1',
       'Sector2TimePercent_1_1', 'Sector3TimePercent_1_1',
       'TyreLifePercent_1_1', 'SpeedI1Percent_1_1', 'SpeedI2Percent_1_1',
       'SpeedFLPercent_1_1', 'LapTime_2', 'Sector1Time_2', 'Sector2Time_2',
       'Sector3Time_2', 'TyreLife_2', 'SpeedI1_2', 'SpeedI2_2', 'SpeedFL_2',
       'LapTimePercent_2', 'Sector1TimePercent_2', 'Sector2TimePercent_2',
       'Sector3TimePercent_2', 'TyreLifePercent_2', 'SpeedI1Percent_2',
      

In [15]:
data[team_cols] = data[team_cols].apply(cap_outliers, axis=1, result_type='expand')
data[individual_cols] = data[individual_cols].apply(cap_outliers, axis=1, result_type='expand')


In [16]:
data['LapTimePrevPercentTeamAverage'] = data[team_cols].apply(np.mean, axis=1)

In [17]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'TyreLife_1', 'SpeedI1_1',
       'SpeedI2_1', 'SpeedFL_1', 'LapTimePercent_1', 'Sector1TimePercent_1',
       'Sector2TimePercent_1', 'Sector3TimePercent_1', 'TyreLifePercent_1',
       'SpeedI1Percent_1', 'SpeedI2Percent_1', 'SpeedFLPercent_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'TyreLife_1_1', 'SpeedI1_1_1', 'SpeedI2_1_1', 'SpeedFL_1_1',
       'LapTimePercent_1_1', 'Sector1TimePercent_1_1',
       'Sector2TimePercent_1_1', 'Sector3TimePercent_1_1',
       'TyreLifePercent_1_1', 'SpeedI1Percent_1_1', 'SpeedI2Percent_1_1',
       'SpeedFLPercent_1_1', 'LapTime_2', 'Sector1Time_2', 'Sector2Time_2',
       'Sector3Time_2', 'TyreLife_2', 'SpeedI1_2', 'SpeedI2_2', 'SpeedFL_2',
       'LapTimePercent_2', 'Sector1TimePercent_2', 'Sector2TimePercent_2',
       'Sector3TimePercent_2', 'TyreLifePercent_2', 'SpeedI1Percent_2',
      

In [18]:
data.to_pickle('train_data_small.pkl')

In [19]:
sched_2023 = fastf1.get_event_schedule(2023)
round_number = 14

In [20]:
sched_2023

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023-02-25 19:30:00,Pre-Season Testing,testing,Practice 1,2023-02-23 10:00:00,Practice 2,2023-02-24 10:00:00,Practice 3,2023-02-25 10:00:00,None,NaT,None,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023-03-05 20:00:00,Bahrain Grand Prix,conventional,Practice 1,2023-03-03 14:30:00,Practice 2,2023-03-03 18:00:00,Practice 3,2023-03-04 14:30:00,Qualifying,2023-03-04 18:00:00,Race,2023-03-05 18:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023-03-19 22:00:00,Saudi Arabian Grand Prix,conventional,Practice 1,2023-03-17 16:30:00,Practice 2,2023-03-17 20:00:00,Practice 3,2023-03-18 16:30:00,Qualifying,2023-03-18 20:00:00,Race,2023-03-19 20:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023-04-02 17:00:00,Australian Grand Prix,conventional,Practice 1,2023-03-31 12:30:00,Practice 2,2023-03-31 16:00:00,Practice 3,2023-04-01 12:30:00,Qualifying,2023-04-01 16:00:00,Race,2023-04-02 15:00:00,True
4,4,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023-04-30 17:00:00,Azerbaijan Grand Prix,sprint_shootout,Practice 1,2023-04-28 13:30:00,Qualifying,2023-04-28 17:00:00,Sprint Shootout,2023-04-29 12:30:00,Sprint,2023-04-29 17:30:00,Race,2023-04-30 15:00:00,True
5,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,Practice 1,2023-05-05 14:00:00,Practice 2,2023-05-05 17:30:00,Practice 3,2023-05-06 12:30:00,Qualifying,2023-05-06 16:00:00,Race,2023-05-07 15:30:00,True
6,6,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2023,2023-05-28 17:00:00,Monaco Grand Prix,conventional,Practice 1,2023-05-26 13:30:00,Practice 2,2023-05-26 17:00:00,Practice 3,2023-05-27 12:30:00,Qualifying,2023-05-27 16:00:00,Race,2023-05-28 15:00:00,True
7,7,Spain,Barcelona,FORMULA 1 AWS GRAN PREMIO DE ESPAÑA 2023,2023-06-04 17:00:00,Spanish Grand Prix,conventional,Practice 1,2023-06-02 13:30:00,Practice 2,2023-06-02 17:00:00,Practice 3,2023-06-03 12:30:00,Qualifying,2023-06-03 16:00:00,Race,2023-06-04 15:00:00,True
8,8,Canada,Montréal,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18 16:00:00,Canadian Grand Prix,conventional,Practice 1,2023-06-16 13:30:00,Practice 2,2023-06-16 16:30:00,Practice 3,2023-06-17 12:30:00,Qualifying,2023-06-17 16:00:00,Race,2023-06-18 14:00:00,True
9,9,Austria,Spielberg,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2023,2023-07-02 17:00:00,Austrian Grand Prix,sprint_shootout,Practice 1,2023-06-30 13:30:00,Qualifying,2023-06-30 17:00:00,Sprint Shootout,2023-07-01 12:00:00,Sprint,2023-07-01 16:30:00,Race,2023-07-02 15:00:00,True


In [21]:
pred_event = sched_2023[sched_2023['RoundNumber'] == round_number]

In [22]:
pred_data = load_practice(pred_event['EventName'].iloc[0], pred_event['EventFormat'].iloc[0], 2023)
for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
    pred_data[col] = pred_event[col].iloc[0]
i = 0
while i < prev_range:
    prev_event = sched_2023[sched_2023['RoundNumber'] == round_number - i - 1]
    prev_data = load_qualifying(prev_event['EventName'].iloc[0], 2023)
    if prev_data.shape[0] != 0:
        prev_data['PrevLapTimePercent' + str(i+1)] = prev_data['LapTimePercent']
        # prev_data['PrevPosition' + str(i+1)] = prev_data['Position']
        prev_data = prev_data.merge(prev_data.groupby('Team')[['LapTimePercent', 'Position']].mean().reset_index(), on=['Team'], suffixes=('', 'PrevTeam' + str(i+1)))

        pred_data = pred_data.merge(prev_data[['DriverNumber', 'PrevLapTimePercent' + str(i+1), 'LapTimePercentPrevTeam' + str(i+1)]], on = 'DriverNumber')
        i += 1

core           INFO 	Loading data for Italian Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info


core        WARNING 	Failed to load data from Ergast API! (This is expected for recent sessions)
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning

In [23]:
pred_data['LapTimePrevPercentTeamAverage'] = pred_data[team_cols].apply(sum, axis=1)

In [24]:
pred_data

,DriverNumber,Team,Driver,LapTime_1,Sector1Time_1,Sector2Time_1,Sector3Time_1,TyreLife_1,SpeedI1_1,SpeedI2_1,...,LapTimePercentPrevTeam1,PrevLapTimePercent2,LapTimePercentPrevTeam2,PrevLapTimePercent3,LapTimePercentPrevTeam3,PrevLapTimePercent4,LapTimePercentPrevTeam4,PrevLapTimePercent5,LapTimePercentPrevTeam5,LapTimePrevPercentTeamAverage
0,1,Red Bull Racing,VER,0 days 00:01:22.657000,0 days 00:00:27.568000,0 days 00:00:27.666000,0 days 00:00:27.423000,8.0,310.0,329.0,...,0.009303,0.000000,0.004130,0.000039,0.002865,0.000000,0.018727,0.000000,0.006103,0.041129
1,10,Alpine,GAS,0 days 00:01:23.931000,0 days 00:00:27.762000,0 days 00:00:28.236000,0 days 00:00:27.933000,9.0,305.0,318.0,...,0.149532,0.070671,0.083391,0.020990,0.018536,0.011174,0.018479,0.012098,0.014295,0.284233
2,11,Red Bull Racing,PER,0 days 00:01:22.834000,0 days 00:00:27.511000,0 days 00:00:28.008000,0 days 00:00:27.315000,15.0,319.0,330.0,...,0.009303,0.008260,0.004130,0.005691,0.002865,0.037454,0.018727,0.012207,0.006103,0.041129
3,14,Aston Martin,ALO,0 days 00:01:23.214000,0 days 00:00:27.441000,0 days 00:00:28.123000,0 days 00:00:27.650000,8.0,312.0,327.0,...,0.074348,0.015777,0.020477,0.005561,0.012799,0.010828,0.018185,0.008076,0.007936,0.133744
4,16,Ferrari,LEC,0 days 00:01:22.966000,0 days 00:00:27.596000,0 days 00:00:27.834000,0 days 00:00:27.536000,9.0,316.0,329.0,...,0.023276,0.007724,0.008496,0.004999,0.009640,0.004797,0.004866,0.000745,0.001848,0.048126
5,2,Williams,SAR,0 days 00:01:23.661000,0 days 00:00:27.419000,0 days 00:00:28.204000,0 days 00:00:28.038000,7.0,321.0,333.0,...,0.049832,0.144742,0.138992,0.034448,0.032287,0.026649,0.017995,0.024180,0.019824,0.258930
6,20,Haas F1 Team,MAG,0 days 00:01:24.217000,0 days 00:00:27.898000,0 days 00:00:28.444000,0 days 00:00:27.875000,12.0,313.0,325.0,...,0.150977,0.075277,0.117691,0.033899,0.020716,0.065244,0.045168,0.024538,0.017697,0.352248
7,22,AlphaTauri,TSU,0 days 00:01:23.271000,0 days 00:00:27.347000,0 days 00:00:28.188000,0 days 00:00:27.736000,2.0,318.0,331.0,...,0.159536,0.065745,0.108182,0.030153,0.024168,0.038111,0.040925,0.021633,0.023109,0.355920
8,23,Williams,ALB,0 days 00:01:23.444000,0 days 00:00:27.481000,0 days 00:00:28.262000,0 days 00:00:27.701000,5.0,318.0,331.0,...,0.049832,0.133242,0.138992,0.030127,0.032287,0.009340,0.017995,0.015468,0.019824,0.258930
9,24,Alfa Romeo,ZHO,0 days 00:01:24.232000,0 days 00:00:27.796000,0 days 00:00:28.460000,0 days 00:00:27.976000,7.0,319.0,328.0,...,0.164333,0.138121,0.109214,0.004725,0.005136,0.039241,0.037367,0.022161,0.021090,0.337141


In [25]:
pred_data.to_pickle('pred_data.pkl')